<a href="https://colab.research.google.com/github/Somsahu20/Langchain/blob/main/YoutubeChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install langchain
!pip install youtube-transcript-api
!pip install pytube
!pip install langchain-community
!pip install langchain-google-genai

In [51]:
from google.colab import userdata
api_key = userdata.get('GEMINI_API_KEY')
yt_key = userdata.get('YT_API')

In [52]:
from youtube_transcript_api import YouTubeTranscriptApi

# Replace this with the actual Video ID from the YouTube URL
# Example: URL is https://www.youtube.com/watch?v=[video_id]
video_id = "iRYZjOuUnlU"

try:
    yt = YouTubeTranscriptApi()
    transcript = yt.fetch(video_id=video_id)

    text = ""

    # for item in transcript:
    #     text += item["text"] + " "

    for item in transcript:
        text += item.text + " "




except Exception as e:
    print(f"Error fetching transcript: {e}")

In [53]:
from langchain_core.documents import Document

doc = Document(
    page_content=text,
    metadata={"topic": "Israel V/S Palestine", "language": "English"}
)


In [54]:
!pip install faiss-cpu

In [55]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001",
    api_key=api_key
)

vector_Stores = FAISS.from_documents(documents=[doc], embedding=embeddings)

retriver = vector_Stores.as_retriever()



In [57]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

template = PromptTemplate(
    template="You are given the context for a youtube transcript:\n {text}.\n From this, you will be asked question. Answer all of the question from the given text only. If the query you receive has some things which are not mentioned in the given text, you just reply back with that you do not know about this. The query is this:\n{query}",
    input_variables=["text", "query"]
)

query = "Who caused the israel and palestine conflict?"
response = retriver.invoke(input=query)

context = ""
for res in response:
  context += res.page_content + ' '

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=api_key,
    temperature=0.2
)

prompt = template.format(text=context, query=query)

result = llm.invoke(prompt)

print(result.content)


The conflict is mostly about two groups of people who claim the same land: Palestinians, who were developing a distinct national identity, and Jews, who were joining the Zionist movement to establish a Jewish nation in their historic homeland in the Middle East.

The conflict began around the early 1900s. Tensions grew as tens of thousands of European Jews moved to the region, which was then under Ottoman rule and later the British Mandate for Palestine. Both Jews and Arabs committed acts of violence as tensions escalated.

The "decades-long Arab-Israeli conflict" officially began after the United Nations approved a plan in 1947 to divide British Palestine into two separate states. The Jews accepted the plan and declared independence as Israel. However, Arab states throughout the region rejected the UN plan, viewing it as European colonialism, and declared war on Israel in 1948.
